# Get California Pizza Kitchen locations

#### Load Python tools and Jupyter config

In [1]:
%load_ext lab_black

In [2]:
import json
import requests
import warnings
import pandas as pd
import altair as alt
from time import sleep
import geopandas as gpd
from random import randint
from vega_datasets import data
from tqdm.notebook import tqdm, trange

In [3]:
pd.options.display.max_rows = 1000
pd.options.display.max_columns = 1000
pd.options.display.max_colwidth = None
warnings.simplefilter(action="ignore")

In [4]:
place = "California Pizza Kitchen"

## Get data

In [5]:
zips = gpd.read_file(
    "../../_reference/data/zips_reference_pop_gen.geojson"
).sort_values("TOTPOP_CY", ascending=False)
zips.columns = zips.columns.str.lower()

In [6]:
zips["latitude"] = zips.centroid.y
zips["longitude"] = zips.centroid.x

In [7]:
zips.head()

,objectid,id,name,state_name,st_abbrev,totpop_cy,popdens_cy,genz_cy,millenn_cy,genx_cy,babyboomcy,oldrgenscy,genbase_cy,geometry,latitude,longitude
5221,27513,77494,Katy,Texas,TX,128751,3204.6,40368,27817,32717,19857,3951,128751,POINT (-95.82624 29.74476),29.744760,-95.826244
4999,26766,77449,Katy,Texas,TX,118772,4405.1,34657,33985,26171,16868,2896,118772,POINT (-95.73768 29.83612),29.836123,-95.737683
2073,6947,60629,Chicago,Illinois,IL,117919,16073.8,35738,33958,22790,17100,4227,117919,POINT (-87.71233 41.77497),41.774969,-87.712328
3666,18676,11368,Corona,New York,NY,117876,56483.8,30015,36378,25974,17142,4767,117876,POINT (-73.85562 40.74959),40.749594,-73.855624
4992,26742,79936,El Paso,Texas,TX,114053,5074.3,29869,32915,22504,21195,4273,114053,POINT (-106.29538 31.76576),31.765758,-106.295381


In [8]:
exclude = [
    "Arkansas"
    "Vermont"
    "Alabama"
    "Nebraska"
    "Utah"
    "Iowa"
    "Mississippi"
    "West Virginia"
    "Oklahoma"
    "District of Columbia"
    "Wyoming"
    "American Samoa"
    "Washington"
    "Guam"
    "South Carolina"
    "Kansas"
    "Maine"
    "Rhode Island"
    "Indiana"
    "Montana"
    "Delaware"
    "North Dakota"
    "Minnesota"
    "Idaho"
    "New Hampshire"
    "Puerto Rico"
    "Alaska"
    "South Dakota"
]

In [9]:
zips_limited = zips[~zips["state_name"].isin(exclude)]

In [10]:
len(zips)

6063

In [11]:
dfs_list = []
response_list = []

for k, v in tqdm(zips_limited.head(2000).iterrows()):
    headers = {
        "authority": "api.cpk.com",
        "accept": "*/*",
        "accept-language": "en-US,en;q=0.9,es;q=0.8",
        "origin": "https://www.cpk.com",
        "referer": "https://www.cpk.com/",
        "sec-ch-ua": '"Not A(Brand";v="99", "Google Chrome";v="121", "Chromium";v="121"',
        "sec-ch-ua-mobile": "?0",
        "sec-ch-ua-platform": '"macOS"',
        "sec-fetch-dest": "empty",
        "sec-fetch-mode": "cors",
        "sec-fetch-site": "same-site",
        "user-agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36",
    }

    params = {
        "lat": f'{v["latitude"]}',
        "long": f'{v["longitude"]}',
        "radius": "500",
        "limit": "20",
    }

    response = requests.get(
        "https://api.cpk.com/api/v1.0/restaurants/near", params=params, headers=headers
    )
    src = pd.DataFrame(pd.DataFrame(response.json()["data"]["restaurants"]))
    dfs_list.append(src)

0it [00:00, ?it/s]

In [12]:
df = (
    pd.concat(dfs_list)[
        [
            "id",
            "name",
            "streetaddress",
            "city",
            "state",
            "zip",
            "slug",
            "latitude",
            "longitude",
            "telephone",
            "utcoffset",
        ]
    ]
    .reset_index(drop=True)
    .drop_duplicates()
)

#### How many?

In [13]:
len(df)

121

#### The result

In [14]:
df.head()

,id,name,streetaddress,city,state,zip,slug,latitude,longitude,telephone,utcoffset
0,29639.0,California Pizza Kitchen Memorial City,"303 Memorial City Mall, #610",Houston,TX,77024,california-pizza-kitchen-memorial-city,29.780900,-95.542595,(713) 365-9473,-6.0
2,29579.0,California Pizza Kitchen Oakbrook,551 Oakbrook Center,Oak Brook,IL,60523,california-pizza-kitchen-oakbrook,41.852289,-87.951811,(630) 571-7800,-6.0
3,29576.0,California Pizza Kitchen Old Orchard,"4999 Old Orchard Road, Suite G-2",Skokie,IL,60077,california-pizza-kitchen-old-orchard,42.061569,-87.747884,(847) 673-1144,-6.0
4,29580.0,California Pizza Kitchen Schaumburg,1550 E. Golf Road,Schaumburg,IL,60173,california-pizza-kitchen-schaumburg,42.051048,-88.040158,(847) 413-9200,-6.0
5,29574.0,California Pizza Kitchen Northbrook Court,"1515 Lake Cook Road, Space 1022",Northbrook,IL,60062,california-pizza-kitchen-northbrook-court,42.150650,-87.817150,(847) 897-5106,-6.0


---

## Geography

#### Make it a geodataframe

In [15]:
df_geo = df.copy()

In [16]:
gdf = gpd.GeoDataFrame(
    df_geo, geometry=gpd.points_from_xy(df_geo.longitude, df_geo.latitude)
)

In [17]:
locations_gdf = gdf.set_crs("EPSG:4326").copy()

---

## Maps

#### US states background

In [18]:
background = (
    alt.Chart(alt.topo_feature(data.us_10m.url, feature="states"))
    .mark_geoshape(fill="#e9e9e9", stroke="white")
    .properties(width=800, height=500, title=f"{place} locations")
    .project("albersUsa")
)

#### Location points map

In [26]:
points = (
    alt.Chart(gdf)
    .mark_circle(size=10, color="#ac6f00")
    .encode(
        longitude="longitude:Q",
        latitude="latitude:Q",
    )
)

point_map = background + points
point_map.configure_view(stroke=None)

alt.LayerChart(...)

#### Location proportional symbols map

In [27]:
symbols = (
    alt.Chart(gdf)
    .transform_aggregate(
        latitude="mean(latitude)",
        longitude="mean(longitude)",
        count="count()",
        groupby=["state"],
    )
    .mark_circle()
    .encode(
        longitude="longitude:Q",
        latitude="latitude:Q",
        size=alt.Size("count:Q", title="Count by state"),
        color=alt.value("#ac6f00"),
        tooltip=["state:N", "count:Q"],
    )
    .properties(title=f"Number of {place} in US, by average lon/lat of locations")
)

symbol_map = background + symbols
symbol_map.configure_view(stroke=None)

alt.LayerChart(...)

---

## Exports

#### JSON

In [21]:
df.to_json(
    f"data/processed/{place.lower().replace(' ', '_')}_locations.json",
    indent=4,
    orient="records",
)

#### CSV

In [22]:
df.to_csv(
    f"data/processed/{place.lower().replace(' ', '_')}_locations.csv", index=False
)

#### GeoJSON

In [23]:
locations_gdf.to_file(
    f"data/processed/{place.lower().replace(' ', '_')}_locations.geojson",
    driver="GeoJSON",
)